# Algoritmo Genético

In [4]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import copy
from mpl_toolkits.mplot3d import axes3d, Axes3D
from matplotlib import cm
import math
import pandas as pd
import random as rnd
from random import random
import functools

# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

## Planteamiento de Problemas

### Plan de Inversión

In [1]:
'''Este problema busca la maxima rentabilidad para una inversion'''
def problema_inversion(parametros):
  x = symbols('x')
  R = -0.001*x**2+0.4*x+3.5
  objetivo = R.subs([[x,parametros[0]]])
  return objetivo
  

## Generar Población Inicial

In [2]:
def poblacionInicial(N,limits): 
  Individuos = list()
  for n in range(N):
    individuo = list()
    for limit in limits:      
      individuo.append(limit[0] + rnd.random()*(limit[1]-limit[0]))  
    Individuos.append(individuo)
  return Individuos

## Métodos de codificación

### Codificación Binaria para Enteros

In [3]:
def int2bin(entero,param={"numbits":10}):
  #print(param["numbits"])
  code="{0:0"+str(param["numbits"])+"b}"
  Xbin=code.format(int(round(entero)))
  return Xbin

def bin2int(Binario,param=False):
  entero=int(Binario, 2)
  return entero

### Codificación Binaria para Números Reales

In [5]:
def BinaryPrecision(Xu,Xl,e):
  n=math.log2((Xu-Xl)/e)
  return int(n)

def Binary2Real(Xu,Xl,n,Xbin):
  Xbin=int(Xbin, 2)
  Value=Xl+((Xu-Xl)/((2**n)-1))*Xbin
  return Value

def Real2Binary(Xu,Xl,n,Value):
  Xbin=(Value-Xl)/((Xu-Xl)/((2**n)-1))
  code="{0:0"+str(n)+"b}"
  Xbin=code.format(int(round(Xbin)))
  return Xbin

def codeBinary2Real(Xbin, param={"Xu":0,"Xl":0,"n":0}):
  Xbin=int(Xbin, 2)
  Value=param["Xl"]+((param["Xu"]-param["Xl"])/((2**param["n"])-1))*Xbin
  return [Value]

def codepoblacionReal2bin(poblacion,limits):
  code=list()
  preV=list()
  for parameter in limits:
    preV.append(BinaryPrecision(parameter[0],parameter[1],parameter[2]))
  for individuo in poblacion:
    indcode=""
    for index,parameter in enumerate(individuo):
      indcode=indcode+Real2Binary(limits[index][0],limits[index][1],preV[index],parameter)
    code.append(indcode)
  return code,preV

In [5]:
#Decodificar la poblacion
def decodificarPob(poblacion,code,codeParameter):
  decoPoblacion = list()   
  for indCode in poblacion: 
    if not isinstance(codeParameter,(bool)):    
      decoDesignP = list()
      c = 0
      for codePar in codeParameter:      
        xbin = indCode[c:c+codePar['n']]
        decoDesignP.append(code(xbin,codePar)[0])
        c = c + codePar['n']
      decoPoblacion.append(decoDesignP)   
    else: decoPoblacion.append(code(indCode,codeParameter))
  return decoPoblacion

In [6]:
def noCodification(XbinIN, param=False):
  return XbinIN

## Cálculo de la Aptitud

### Aptitud Proporcional

In [2]:
def AptitudProporcional(objective,population,code,codeParam=False,case="minimo"):   
  aptitud=list()
  #Decodificar la poblacion
  decoPoblacion = decodificarPob(population,code,codeParam)  

  for individuo in decoPoblacion: #decoPoblacion
    if case == "minimo":
      aptitud.append(1/objective([individuo])) #code(individuo,codeParam)
    elif case == "maximo":
      aptitud.append(objective([individuo])) #code(individuo,codeParam)
    else:
      aptitud.append(1/objective([individuo])) #code(individuo,codeParam)
    
  fS=sum(aptitud)
  prob=list()
  for app in aptitud:
    prob.append(float(app/fS))
  return aptitud,prob

## Selección proporcional

### Selección Disruptiva

In [8]:
def seleccionDisruptiva(poblacion,aptitud,parametros=False):
  n = len(poblacion)
  padres = list()
  expected = list()
  aptitudNormalizada = list()
  aptitudMedia = float(sum(aptitud)/n)  
  for apt in aptitud:
    aptitudNormalizada.append(abs(apt-aptitudMedia))
  mediaNormalizada = float(sum(aptitudNormalizada)/n)
  
  for nApt in aptitudNormalizada:
    expected.append(nApt/mediaNormalizada)
  # print(expected)
  roundExpected = [round(e) for e in expected]
  # print(sum(roundExpected))
  i = 0
  while len(padres) < n:#for i in range(n):
    for j in range(roundExpected[i]):
      if len(padres) < n: padres.append(poblacion[i])
    if i == n-1 and len(padres) < n: 
      mayor = np.argmax(roundExpected)
      if mayor > 1: 
        while len(padres) < n: padres.append(poblacion[mayor])    
      else: i = 0            
    else: i += 1
  rnd.shuffle(padres)
  # print(len(padres))
  return padres

### Selección por Valor Esperado

In [9]:
def ValorEsperado(Poblacion,Probabilidades,parametros=False):
  Expected=list()
  padres=list()
  n = len(Poblacion)
  for proba in Probabilidades:
    Expected.append(proba*len(Poblacion))
  # print(Expected)
  roundExpected = [round(e) for e in Expected]
  # print(roundExpected,sum(roundExpected))
  i = 0
  while len(padres) < n:#for i in range(len(Poblacion)):
    #print(round(Expected[i]))        
    for j in range(roundExpected[i]):
      if len(padres) < n: padres.append(Poblacion[i])
    if i == n-1 and len(padres) < n: 
      mayor = np.argmax(roundExpected)
      if mayor > 1: 
        while len(padres) < n: padres.append(Poblacion[mayor])    
      else: i = 0            
    else: i += 1
  
  rnd.shuffle(padres)
  # print(len(padres))
  return padres

### Selección por Ruleta

In [10]:
def Ruleta(Poblacion,Probabilidades,parametros=False):
  padres=list()
  #print(len(Probabilidades))
  for i in range(len(Probabilidades)):
    meta=random()
    index=0
    suma=Probabilidades[index]
    #print("meta")
    #print(meta)
    #print("sumatoria")
    while suma<=meta:
      #print(suma)
      index=index+1
      suma=suma+Probabilidades[index]
    #print("indice")
    #print(index)
    padres.append(Poblacion[index])
  return(padres)

### Selección por Sobrante Estocástico

In [11]:
def SobranteEstocastico(Poblacion,Probabilidades,parametros=False):
  # Calcular el valor esperado considerando solo los enteros 
  Expected=list()
  restante=list()
  padres=list()
  for proba in Probabilidades:
    Expected.append(proba*len(Poblacion))
  #print(Expected)
  for i in range(len(Poblacion)):
    #print(math.trunc(Expected[i]))
    restante.append(Expected[i]-math.trunc(Expected[i]))
    for j in range(math.trunc(Expected[i])):
      padres.append(Poblacion[i])
  #print(restante)
  
  # Calculo de las nuevas probabilidades
  fS=sum(restante)
  prob=list()
  if  fS!=0:
    for app in restante:
      prob.append(float(app/fS))
  
  
    #print(prob)
    # Seleccion por ruleta
    for i in range(len(Probabilidades)-len(padres)):
      meta=random()
      index=0
      suma=Probabilidades[index]
      while suma<=meta:
        #print(suma)
        index=index+1
        suma=suma+Probabilidades[index]
      #print(index)
      padres.append(Poblacion[index])
  rnd.shuffle(padres)    
  return padres

### Selección por Universal Estocástico

In [12]:
def universalEstocastico(Poblacion,Probabilidades,parametros=False):
  padres = list()
  Expected=list()
  for proba in Probabilidades:
    Expected.append(proba*len(Poblacion))
  ptr = rnd.random()
  # ptr=0.4
  index = 0
  suma = 0
  while len(padres)<len(Poblacion):
    suma = suma + Expected[index] 
    while suma > ptr:
      padres.append(Poblacion[index])
      ptr += 1
    index += 1

  rnd.shuffle(padres)
  return padres

### Selección mediante Torneo

In [13]:
def torneo(Poblacion,Probabilidades,parametros={"p":2}):
  baraja=list(range(len(Poblacion)))
  padres=list()
  for i in range(parametros["p"]):
    rnd.shuffle(baraja)
    n=0
    while n<len(Poblacion): 
      torneo=baraja[n:n+parametros["p"]]
      win=max([Probabilidades[i] for i in torneo])
      Iwin=Probabilidades.index(win)
      padres.append(Poblacion[Iwin])
      
      n=n+parametros["p"]
      
      #print(n)
      #print(torneo)
      #print(win)
      #print(Iwin)
      
  return padres

## Técnicas de Cruza

### Cruza en 1 punto

In [14]:
def cruza1Punto(padres,param=False):
  hijos=list()
  # print(len(padres))
  i=0
  while i<len(padres):
    pareja=copy.deepcopy(padres[i:i+2])
    cruze=rnd.randint(1,len(padres[i])-1)
    #print(cruze)
    hijo1=pareja[0][:cruze]+pareja[1][cruze:]
    hijo2=pareja[1][:cruze]+pareja[0][cruze:]
    i=i+2
    hijos.append(hijo1)
    hijos.append(hijo2)
    #print(pareja)
    #print(cruze)
    #print(hijo1)
    #print(hijo2)
  return hijos

### Cruza en N puntos

In [15]:
def cruzaNPunto(padres,param={"n":2}):
  hijos=list()
  i=0
  while i<len(padres):
    pareja=copy.deepcopy(padres[i:i+2])
    #print(pareja)
    hijo1=copy.deepcopy(pareja[0])
    hijo2=copy.deepcopy(pareja[1])
    for j in range(param['n']):
      cruze=rnd.randint(1,len(padres[i])-1)
      #print(cruze)
      hijoA=hijo1[:cruze]+hijo2[cruze:]
      hijoB=hijo2[:cruze]+hijo1[cruze:]
      hijo1=hijoA
      hijo2=hijoB
      #print(hijo1)
      #print(hijo2)
    
    i=i+2
    hijos.append(hijo1)
    hijos.append(hijo2)
    
    
    
  return hijos

### Cruza Uniforme

In [16]:
def cruzaUniforme(padres,param=False):
  hijos=list()
  i=0
  while i<len(padres):
    pareja=copy.deepcopy(padres[i:i+2])
    hijo1=""
    hijo2=""
    for j in range(len(padres[0])):
      selec=rnd.randint(0,1)
      #print(selec)
      hijo1=hijo1+pareja[selec][j]
      hijo2=hijo2+pareja[abs(selec-1)][j]
    i=i+2
    hijos.append(hijo1)
    hijos.append(hijo2)
    #print(pareja)
    #print(hijo1)
    #print(hijo2)
  return hijos

### Order Cross Over

In [17]:
def orderCrossOver(padres,parametros=False):
  n = len(padres)
  m = len(padres[0])
  hijos = list()
  k = 0
  while k < n:
    P1 = copy.deepcopy(padres[k:k+1][0])     
    P2 = copy.deepcopy(padres[k+1:k+2][0])
    # print(P1)
    # print(P2)       
    hijo1 = [None]*m
    hijo2= [None]*m    
    r1 = rnd.randint(0,m-1)
    # rep = True
    # while rep:
    r2 = rnd.randint(0,m-1)
      # if r1 != r2: rep = False
    R = sorted([r1,r2])        
    # subChain = pareja[0][R[0]:R[1]]       
    hijo1[R[0]:R[1]] = P1[R[0]:R[1]] 
    hijo2[R[0]:R[1]] = P2[R[0]:R[1]] 
    #Producir Hijo 1
    for i in hijo1:
      if i != None: P2.remove(i)        
    for i in P2:
      for index,j in enumerate(hijo1):
        if j == None: 
          hijo1[index] = i
          break
    #Producir hijo 2
    for i in hijo2:
      # print(i)
      # print(pareja[0])
      if i != None: P1.remove(i)    
    for i in P1:
      for index,j in enumerate(hijo2):
        if j == None: 
          hijo2[index] = i
          break      
    
    hijos.append(hijo1)
    hijos.append(hijo2)
    k += 2
  return hijos

## Mutación

### Mutación Binaria

In [18]:
def mutacionBinaria(individuos,parameter={"umbral":0.1}):
  hijos=copy.deepcopy(individuos)
  for elem,individuo in enumerate(individuos):
    for index,gen in enumerate(individuo):
      if rnd.random()<parameter["umbral"]:
        mutacion=str(abs(int(gen)-1))
        hijos[elem]=individuo[:index]+mutacion+individuo[index+1:]
  return hijos

### Mutación por Inserción

In [19]:
def mutacionInsercion(individuos,parameter={"umbral":0.1}):
  hijos=copy.deepcopy(individuos)
  for elem,hijo in enumerate(hijos):
    excepciones=list()
    for index,gen in enumerate(hijo):      
      if random()<parameter["umbral"] and gen not in excepciones :
        posArbitraria=rnd.randint(0,len(individuos[0]))
        genMutado = hijo.pop(index)        
        hijo.insert(posArbitraria, genMutado ) 
        excepciones.append(genMutado)    
  return hijos

## Algoritmo Genético Integrado

In [1]:
def algoritmoGenetico(problema,poblacion,code,NIteracion,Aptitud=AptitudProporcional,
                      Seleccion=SobranteEstocastico,cruza=cruza1Punto,mutacion=mutacionBinaria,
                      SelecParameter=False ,cruzaParameter=False,mutacionParameters=False,
                      codeParameter=False,case="minimo"):
  print('Realizando optimizacion con GA...')

  histSolucion=list()
  bestApt = None
  for i in range(NIteracion):
    aptitud,probabilidad=Aptitud(problema,poblacion,code,codeParameter,case)
    bestAptG = max(aptitud)
    if not histSolucion or bestAptG >= bestApt:
        bestApt = bestAptG
        bestIndex = aptitud.index(bestApt)
        bestSol = poblacion[bestIndex]
        histSolucion.append((bestSol,bestApt))
#     Historico.append(aptitud)
    padres=Seleccion(poblacion,probabilidad,SelecParameter) #aptitud    
    hijos=cruza(padres,cruzaParameter)
    hijos=mutacion(hijos,mutacionParameters)
    poblacion=copy.deepcopy(hijos)
  decodePop= decodificarPob(poblacion,code,codeParameter)
  print('Optimizacion finalizada')
  return decodePop,histSolucion #,Historico

NameError: name 'AptitudProporcional' is not defined

## Resolviendo Problemas

### Resolver Problema Plan de Inversión

In [ ]:
# poblacion = poblacionInicial(20,[[50,300]])
# # poblacion = [[22.359848097498308], [18.000490966289647], [19.34882629636717], [24.188415423903447], [38.92030251982626], [3.1392107782161505], [2.396101873801153], [2.6368852275487953], [38.28085839504099], [25.98472027097023], [12.9058383727712], [21.76198207366142], [7.477971663163396], [6.026557762291804], [31.100226578019175], [32.52030939941475], [26.737437715067024], [22.466419599701073], [23.05287224161532], [13.200565110778655]]
# binPoblacion, n = codepoblacionReal2bin(poblacion,[[300,50,0.01]])
# print(poblacion)
# print(n)

In [70]:
# Poblacion, decoPoblacion, Historial = algoritmoGenetico(problema_inversion,binPoblacion,codeBinary2Real,30,
#                                                         Aptitud=AptitudProporcional,Seleccion=torneo,
#                                                         cruza=cruzaNPunto,mutacion=mutacionBinaria,SelecParameter={"p":4}, #{"p":2}
#                                                         cruzaParameter={"n":3},mutacionParameters={'umbral':0.01},
#                                                         codeParameter=[{'Xu':300,'Xl':50,'n':n[0]}],case='maximo')

# print("la poblacion final:  ")
# print(Poblacion)
# print(decoPoblacion)

[36.7496829997067, 43.3956526766100, 26.6101646490446, 37.8419604234570, 40.0829518341107, 29.6368366285120, 38.6218719213705, 43.3885458501389, 41.8304428257602, 38.5876755445860, 33.0752506619501, 21.1463886810590, 41.2569637344020, 25.4474639498762, 38.3659903356716, 41.3258700905565, 30.3748596551075, 34.4070818868851, 30.5978826195930, 40.2911042121145]
[38.2036321974833, 43.4469651581562, 41.2569637344020, 40.0657879523644, 37.6042221476901, 30.5978826195930, 38.4843697594620, 43.4100932702295, 41.5477941843943, 43.4341728705081, 39.8505645806280, 41.6668544302165, 37.9006305606438, 31.4708636836628, 43.2414285700635, 41.0679579830086, 43.4100932702295, 41.2569637344020, 39.1849356300685, 43.4590049582955]
[43.4455318486158, 41.3258700905565, 43.0141056769569, 42.9745463336420, 43.4100932702295, 43.4100932702295, 43.4469651581562, 41.0679579830086, 43.3669506530636, 42.0799700724968, 43.0248196657713, 42.8810945516082, 41.6668544302165, 43.4341728705081, 43.4100932702295, 43.4100